# Read ASC files

## Import libraries

In [1]:
import glob, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
#import rasterio as rio
from osgeo import gdal
import numpy as np
from osgeo import osr

## Read DENUE file

In [2]:
gpkg_gdf = gpd.read_file('data/colima/agebs.gpkg')
gpkg_gdf = gpkg_gdf[["CVE_AGEB", "geometry"]]
gpkg_gdf.head()

,CVE_AGEB,geometry
0,0676,"POLYGON ((2319182.093 807954.424, 2319286.446 ..."
1,0727,"POLYGON ((2318544.164 807545.878, 2318566.020 ..."
2,1142,"POLYGON ((2316882.335 805042.325, 2317096.142 ..."
3,0411,"POLYGON ((2319801.226 808675.655, 2319836.062 ..."
4,1068,"POLYGON ((2320222.167 809638.735, 2320222.724 ..."


## GDAL

## Ubicación de los archivos ASC

In [3]:
asc_files = glob.glob('data/Rasters/*/*.asc')
asc_files[:5]

['data/Rasters\\Hydraulic\\Depth____0.0.asc',
 'data/Rasters\\Hydraulic\\Depth____10200.0.asc',
 'data/Rasters\\Hydraulic\\Depth____10500.0.asc',
 'data/Rasters\\Hydraulic\\Depth____10800.0.asc',
 'data/Rasters\\Hydraulic\\Depth____11100.0.asc']

## GDAL

In [18]:
# Open the ASC file as a GDAL dataset
asc_ds = gdal.Open(asc_files[0])

### first method

In [10]:
# Get the geotransform and projection of the ASC file
asc_gt = asc_ds.GetGeoTransform()
asc_proj = asc_ds.GetProjection()

# Read the ASC file as a numpy array
asc_arr = asc_ds.ReadAsArray()

# Close the GDAL dataset
asc_ds = None

# Create a GeoDataFrame from the ASC array, geotransform and projection
asc_gdf = gpd.GeoDataFrame.from_features(
    gdal.polygonize(asc_arr, None, options=["8CONNECTED=8"]),
    crs=asc_proj
)

# Set the column name of the ASC values
asc_gdf.columns = ["value", "geometry"]

# Open the gpkg file as a GeoDataFrame
#gpkg_gdf = gpd.read_file("file.gpkg")

# Perform a spatial join to extract the zones that intersect with the ASC file
zones_gdf = gpd.sjoin(asc_gdf, gpkg_gdf, how="inner", op="intersects")

# Save the zones as a new gpkg file
zones_gdf.to_file("zones.gpkg", driver="GPKG")

AttributeError: module 'osgeo.gdal' has no attribute 'polygonize'

### second method

In [5]:
# Get the spatial reference of the dataset and extract the projection information:
proj_info = asc_ds.GetProjection()
spatial_ref = osr.SpatialReference()
spatial_ref.ImportFromWkt(proj_info)

5

In [19]:
asc_ds.GetProjection()

''

In [20]:
# Extract the geotransform of the dataset:
geotransform = asc_ds.GetGeoTransform()
geotransform

(631151.0, 5.0, 0.0, 2134796.75, 0.0, -5.0)

In [7]:
# Read the data from the ASC file and convert it to a numpy array:
data = asc_ds.ReadAsArray()

In [9]:
# Identify the number of rows and columns in the dataset:
rows, cols = data.shape
rows, cols

(1923, 1540)

In [11]:
# Compute the x and y coordinates for each cell in the dataset:

x_coords = np.arange(cols) * geotransform[1] + geotransform[0]
y_coords = np.arange(rows) * geotransform[5] + geotransform[3]
x_coords, y_coords

(array([631151., 631156., 631161., ..., 638836., 638841., 638846.]),
 array([2134796.75, 2134791.75, 2134786.75, ..., 2125196.75, 2125191.75,
        2125186.75]))

In [16]:
data

array([[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       ...,
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.]],
      dtype=float32)

In [15]:
gpkg_gdf

,CVE_AGEB,geometry
0,0676,"POLYGON ((2319182.093 807954.424, 2319286.446 ..."
1,0727,"POLYGON ((2318544.164 807545.878, 2318566.020 ..."
2,1142,"POLYGON ((2316882.335 805042.325, 2317096.142 ..."
3,0411,"POLYGON ((2319801.226 808675.655, 2319836.062 ..."
4,1068,"POLYGON ((2320222.167 809638.735, 2320222.724 ..."
5,0708,"POLYGON ((2319089.306 806584.763, 2319031.093 ..."
6,1458,"POLYGON ((2317691.004 805411.285, 2317691.315 ..."
7,0318,"POLYGON ((2320007.377 807574.674, 2320007.120 ..."
8,0680,"POLYGON ((2318612.394 807783.209, 2318620.778 ..."
9,0712,"POLYGON ((2319624.770 806222.448, 2319536.796 ..."
